In [1]:
# Simple OpenAI tools + tool_choice demo, similar to your current notebook logic.
# - Two tools: Channel1 (Bollywood), Channel2 (Cricket)
# - Counters: Bviews, Cviews
# - chat_run(message, history, tool_choice) supports: "auto", "none", or force a specific tool by name

import json
from openai import OpenAI

# Initialize client
client = OpenAI()

# Global counters
Bviews = 0
Cviews = 0

# Tools: implement simple behavior (can also call LLM inside if desired)
def Channel1():
    global Bviews
    Bviews += 1
    return "Bollywood: Latest trends, box office updates, and celeb news."

def Channel2():
    global Cviews
    Cviews += 1
    return "Cricket: Live scores, match analysis, and player stats."

def unknownQuestion():
    return "I only handle Bollywood or Cricket queries."

# Tool schema for the API
tools = [
    {"type": "function", "function": {"name": "Channel1", "description": "Bollywood info", "parameters": {"type": "object", "properties": {}, "required": []}}},
    {"type": "function", "function": {"name": "Channel2", "description": "Cricket info", "parameters": {"type": "object", "properties": {}, "required": []}}},
    {"type": "function", "function": {"name": "unknownQuestion", "description": "Fallback for unknown topics", "parameters": {"type": "object", "properties": {}, "required": []}}},
]

# Simple system prompt like yours
SystemPrompt = (
    "You have two tools. Use Channel1 for Bollywood, Channel2 for Cricket. "
    "If it's neither, use unknownQuestion. Provide concise answers."
)

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments or "{}")
        tool_fn = globals().get(tool_name)
        result_text = tool_fn(**arguments) if tool_fn else ""
        results.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps({"result": result_text})
        })
    return results

def chat_run(message, history=None, tool_choice="auto"):
    """
    - message: current user string
    - history: list of dicts like [{"role":"user","content":...}, {"role":"assistant","content":...}, ...]
    - tool_choice:
        - "auto" (default): model decides tools
        - "none": model not allowed to call tools
        - {"type":"function","function":{"name":"Channel1"}}: force a specific tool
    """
    if history is None:
        history = []

    messages = [{"role": "system", "content": SystemPrompt}] + history + [{"role": "user", "content": message}]

    # Map simple string choices to API-compatible values
    if tool_choice == "auto":
        tool_choice_arg = "auto"
    elif tool_choice == "none":
        tool_choice_arg = "none"
    elif isinstance(tool_choice, dict):
        tool_choice_arg = tool_choice
    else:
        tool_choice_arg = "auto"

    while True:
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
            tool_choice=tool_choice_arg
        )
        choice = resp.choices[0]
        finish = choice.finish_reason

        if finish == "tool_calls":
            tool_calls = choice.message.tool_calls or []
            tool_results = handle_tool_calls(tool_calls)
            messages.append(choice.message)   # assistant with tool_calls
            messages.extend(tool_results)     # append each tool result
            # after tools executed once, let the model decide follow-up
            tool_choice_arg = "auto"
            continue

        # Final assistant message
        final_msg = choice.message
        return final_msg.content

# -------------------------
# Demo usage (prints only):
# -------------------------

# 1) Let the model pick tools automatically
print("AUTO:")
print(chat_run("What's the latest in Bollywood?"))
print(chat_run("Who won the last cricket match?"))
print(f"Bviews={Bviews}, Cviews={Cviews}")

# 2) Force NO tools (plain model response)
print("\nNONE:")
print(chat_run("Cricket update please", tool_choice="none"))
print(f"Bviews={Bviews}, Cviews={Cviews}  # unchanged because tools were not called")

# 3) Force a specific tool
print("\nFORCE Channel1:")
print(chat_run("Talk about anything", tool_choice={"type": "function", "function": {"name": "Channel1"}}))
print(f"Bviews={Bviews}, Cviews={Cviews}")

AUTO:
The latest in Bollywood includes updates on trends, box office hits, and celebrity news.
I currently don't have the latest information about the last cricket match. Please check a sports news website or app for the most recent updates.
Bviews=1, Cviews=1

NONE:
{"tool_uses":[{"recipient_name":"functions.Channel2","parameters":{}}]}
Bviews=1, Cviews=1  # unchanged because tools were not called

FORCE Channel1:
None
Bviews=1, Cviews=1


In [5]:
# Minimal router: detect topic in Python, call the right "channel" function,
# each channel calls the LLM directly and returns the message.

import re
from openai import OpenAI

client = OpenAI()

Bviews = 0
Cviews = 0

def channel1_bollywood(question: str) -> str:
    global Bviews
    Bviews += 1
    prompt = f"""You are a concise Bollywood assistant.
Answer clearly and briefly about Bollywood only.

User question: {question}
"""
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You answer Bollywood questions."},
                  {"role": "user", "content": prompt}]
    )
    return resp.choices[0].message.content

def channel2_cricket(question: str) -> str:
    global Cviews
    Cviews += 1
    prompt = f"""You are a concise Cricket assistant.
Answer clearly and briefly about cricket only.

User question: {question}
"""
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You answer Cricket questions."},
                  {"role": "user", "content": prompt}]
    )
    return resp.choices[0].message.content

def unknown_fallback(question: str) -> str:
    return "I can help with Bollywood or Cricket only. Please ask about one of those."

BOLLYWOOD_KWS = re.compile(r"\b(bollywood|movie|film|actor|actress|box\s*office|bolly)\b", re.I)
CRICKET_KWS   = re.compile(r"\b(cricket|ipl|odi|t20|test\s*match|batsman|bowler|wicket)\b", re.I)

def detect_topic(question: str) -> str:
    if BOLLYWOOD_KWS.search(question or ""):
        return "bollywood"
    if CRICKET_KWS.search(question or ""):
        return "cricket"
    return "unknown"

def chat_run(question: str) -> str:
    topic = detect_topic(question)
    if topic == "bollywood":
        return channel1_bollywood(question)
    if topic == "cricket":
        return channel2_cricket(question)
    return unknown_fallback(question)

# Example usage:
if __name__ == "__main__":
    print(chat_run("What's the latest Bollywood box office news?"))
    print(chat_run("Who scored most runs in the last ODI?"))
    print(f"Bviews={Bviews}, Cviews={Cviews}")

As of October 2023, the latest Bollywood box office updates include significant earnings from films like "Jawan" and "Pathaan," which have broken records. The latest releases continue to perform well, with new films entering the box office rankings weekly. Please check current sources for the most up-to-date figures.
I don’t have real-time data access. Please check the latest sports news or websites for the most recent ODI match details.
Bviews=1, Cviews=1


In [ ]:
# Model routes to tools; tools call LLM and return the final text.
# No Gradio.

import json
from openai import OpenAI
client = OpenAI()

Bviews = 0
Cviews = 0

def Channel1():
    global Bviews
    Bviews += 1
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You answer Bollywood questions concisely."},
            {"role": "user", "content": "Provide the answer about Bollywood for the user's last question."}
        ]
    )
    return resp.choices[0].message.content

def Channel2():
    global Cviews
    Cviews += 1
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You answer Cricket questions concisely."},
            {"role": "user", "content": "Provide the answer about Cricket for the user's last question."}
        ]
    )
    return resp.choices[0].message.content

def unknownQuestion():
    return "I only handle Bollywood or Cricket. Please ask about one of those."

tools = [
    {"type": "function", "function": {"name": "Channel1", "description": "Bollywood info", "parameters": {"type": "object", "properties": {}, "required": []}}},
    {"type": "function", "function": {"name": "Channel2", "description": "Cricket info",  "parameters": {"type": "object", "properties": {}, "required": []}}},
    {"type": "function", "function": {"name": "unknownQuestion", "description": "Fallback", "parameters": {"type": "object", "properties": {}, "required": []}}},
]

SYSTEM = "Choose exactly one tool: Channel1 for Bollywood, Channel2 for Cricket, otherwise unknownQuestion. Then provide the final answer."

def handle_tool_calls(tool_calls):
    results = []
    for tc in tool_calls:
        fn = globals().get(tc.function.name)
        out = fn(**(json.loads(tc.function.arguments or "{}"))) if fn else ""
        results.append({"role": "tool", "tool_call_id": tc.id, "content": json.dumps({"result": out})})
    return results

def ask(user_question: str, tool_choice="auto"):
    messages = [
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": user_question}
    ]
    choice_arg = tool_choice  # "auto" lets the model pick the correct tool

    while True:
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
            tool_choice=choice_arg
        )
        ch = resp.choices[0]
        if ch.finish_reason == "tool_calls":
            # Execute selected tool(s)
            messages.append(ch.message)
            messages.extend(handle_tool_calls(ch.message.tool_calls))
            # After tools run once, switch to auto to let model produce final text
            choice_arg = "auto"
            continue
        return ch.message.content

# Demo
print(ask("What's the latest Bollywood box office update?"))
print(ask("Who is the best ODI batsman right now?"))
print("Bviews =", Bviews, "Cviews =", Cviews)

I can provide information about Bollywood or Cricket. Please specify which one you'd like to know about!
I can only provide information about Bollywood or Cricket. Please specify if you'd like to know about one of those topics.
Bviews = 1 Cviews = 0


In [7]:
import gradio as gr

demo = gr.Interface(
    fn=lambda q: ask(q, tool_choice="auto"),
    inputs=gr.Textbox(label="Your question", lines=2, placeholder="Ask about Bollywood or Cricket"),
    outputs=gr.Textbox(label="Answer"),
    title="Bollywood/Cricket Router (Tools)",
    description="The model picks Channel1 (Bollywood) or Channel2 (Cricket) automatically."
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [ ]:
# 1) Update tool functions to accept the query and use it
def Channel1(query: str):
    global Bviews
    Bviews += 1
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Answer Bollywood questions concisely and factually."},
            {"role": "user", "content": query}
        ]
    )
    return resp.choices[0].message.content

def Channel2(query: str):
    global Cviews
    Cviews += 1
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Answer Cricket questions concisely and factually."},
            {"role": "user", "content": query}
        ]
    )
    return resp.choices[0].message.content

def unknownQuestion(query: str):
    return "I can help with Bollywood or Cricket only. Please ask about one of those."

In [9]:
# 2) Update tool schemas to accept a 'query' string
tools = [
    {"type": "function", "function": {
        "name": "Channel1",
        "description": "Bollywood info (movies, box office, awards).",
        "parameters": {
            "type": "object",
            "properties": {"query": {"type": "string", "description": "User question"}},
            "required": ["query"]
        }}},
    {"type": "function", "function": {
        "name": "Channel2",
        "description": "Cricket info (players, matches, formats: IPL, ODI, T20, Tests,records statistics).",
        "parameters": {
            "type": "object",
            "properties": {"query": {"type": "string", "description": "User question"}},
            "required": ["query"]
        }}},
    {"type": "function", "function": {
        "name": "unknownQuestion",
        "description": "Fallback for non-Bollywood/Cricket topics.",
        "parameters": {
            "type": "object",
            "properties": {"query": {"type": "string", "description": "User question"}},
            "required": ["query"]
        }}},
]

In [10]:
# 3) Ensure you pass arguments into the tool functions
def handle_tool_calls(tool_calls):
    results = []
    for tc in tool_calls:
        fn = globals().get(tc.function.name)
        args = json.loads(tc.function.arguments or "{}")
        out = fn(**args) if fn else ""
        results.append({"role": "tool", "tool_call_id": tc.id, "content": json.dumps({"result": out})})
    return results

In [11]:
# 4) Nudge routing with a clearer system instruction
SYSTEM = (
    "Choose exactly one tool based on the user question: "
    "Channel1 for Bollywood topics , "
    "Channel2 for Cricket topics (players, matches, IPL/ODI/T20/Tests), "
    "otherwise unknownQuestion. Pass the user's query string as 'query'. "
    "After tools, provide the final concise answer."
)

In [12]:


def ask(user_question: str, tool_choice="auto"):
    messages = [
    ]
    choice_arg = tool_choice  # "auto" lets the model pick the correct tool

    while True:
        resp = client.chat.completions.create(
         {"role": "system", "content": SYSTEM},
        {"role": "user", "content": user_question}
           model="gpt-4o-mini",
            messages=messages,
            tools=tools,
            tool_choice=choice_arg
        )
        ch = resp.choices[0]
        if ch.finish_reason == "tool_calls":
            # Execute selected tool(s)
            messages.append(ch.message)
            messages.extend(handle_tool_calls(ch.message.tool_calls))
            # After tools run once, switch to auto to let model produce final text
            choice_arg = "auto"
            continue
        return ch.message.content



SyntaxError: invalid syntax. Perhaps you forgot a comma? (2348459670.py, line 9)